In [16]:
# =============================================
# Artificial Neural Network Classification with Hyperparameter Tuning
# Dataset: Alphabets_data.csv
# =============================================

# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from keras.optimizers import Adam # Corrected import

# =============================================
# 1. Data Exploration and Preprocessing
# =============================================

# Load Dataset
df = pd.read_csv("Alphabets_data.csv")  # ensure correct file path

print("Dataset Shape:", df.shape)
print(df.head())

# Handle Missing Values
df = df.dropna()

# Separate features and target
X = df.drop('letter', axis=1) # Changed 'label' to 'letter'
y = df['letter'] # Changed 'label' to 'letter'

# Encode Target Labels
le = LabelEncoder()
y = le.fit_transform(y)

# Normalize Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# =============================================
# 2. Build and Train Basic ANN Model
# =============================================

def build_model(neurons=64, activation='relu', learning_rate=0.001, layers=1):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train.shape[1], activation=activation))
    for _ in range(layers - 1):
        model.add(Dense(neurons, activation=activation))
        model.add(Dropout(0.2))
    model.add(Dense(len(np.unique(y)), activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Train Basic Model
basic_model = build_model()
history = basic_model.fit(X_train, y_train, epochs=20, batch_size=32,
                          validation_split=0.2, verbose=1)

# Evaluate Basic Model
y_pred_basic = np.argmax(basic_model.predict(X_test), axis=1)
acc_basic = accuracy_score(y_test, y_pred_basic)
print("\n=== Basic Model Evaluation ===")
print(f"Accuracy: {acc_basic:.4f}")
print(classification_report(y_test, y_pred_basic))

# =============================================
# 3. Simple Manual Hyperparameter Tuning
# =============================================

# Try different combinations manually
params = [
    {'neurons': 32, 'layers': 1, 'lr': 0.001},
    {'neurons': 64, 'layers': 2, 'lr': 0.001},
    {'neurons': 128, 'layers': 2, 'lr': 0.005},
    {'neurons': 64, 'layers': 3, 'lr': 0.01},
]

best_acc = 0
best_model = None
best_params = None

for p in params:
    print(f"\nTraining model with params: {p}")
    model = build_model(neurons=p['neurons'], layers=p['layers'], learning_rate=p['lr'])
    model.fit(X_train, y_train, epochs=15, batch_size=32, verbose=0)
    y_pred = np.argmax(model.predict(X_test), axis=1)
    acc = accuracy_score(y_test, y_pred)
    print(f"Validation Accuracy: {acc:.4f}")

    if acc > best_acc:
        best_acc = acc
        best_model = model
        best_params = p

print("\nBest Hyperparameters Found:", best_params)
print("Best Validation Accuracy:", best_acc)

Dataset Shape: (20000, 17)
  letter  xbox  ybox  width  height  onpix  xbar  ybar  x2bar  y2bar  xybar  \
0      T     2     8      3       5      1     8    13      0      6      6   
1      I     5    12      3       7      2    10     5      5      4     13   
2      D     4    11      6       8      6    10     6      2      6     10   
3      N     7    11      6       6      3     5     9      4      6      4   
4      G     2     1      3       1      1     8     6      6      6      6   

   x2ybar  xy2bar  xedge  xedgey  yedge  yedgex  
0      10       8      0       8      0       8  
1       3       9      2       8      4      10  
2       3       7      3       7      3       9  
3       4      10      6      10      2       8  
4       5       9      1       7      5      10  
Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2904 - loss: 2.6392 - val_accuracy: 0.6528 - val_loss: 1.3906
Epoch 2/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6906 - loss: 1.2123 - val_accuracy: 0.7344 - val_loss: 0.9989
Epoch 3/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7505 - loss: 0.9161 - val_accuracy: 0.7656 - val_loss: 0.8383
Epoch 4/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7848 - loss: 0.7675 - val_accuracy: 0.7847 - val_loss: 0.7419
Epoch 5/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8133 - loss: 0.6775 - val_accuracy: 0.8091 - val_loss: 0.6701
Epoch 6/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8295 - loss: 0.6105 - val_accuracy: 0.8266 - val_loss: 0.6159
Epoch 7/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8398 - loss: 0.5816 - val_accuracy: 0.8359 - val_loss: 0.5769
Epoch 8/20
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8547 - loss: 0.5349 - val_accuracy: 0.8425 - val_

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Validation Accuracy: 0.8688

Training model with params: {'neurons': 64, 'layers': 2, 'lr': 0.001}


...

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Validation Accuracy: 0.9247

Training model with params: {'neurons': 128, 'layers': 2, 'lr': 0.005}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Validation Accuracy: 0.9545

Training model with params: {'neurons': 64, 'layers': 3, 'lr': 0.01}


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Validation Accuracy: 0.8665

Best Hyperparameters Found: {'neurons': 128, 'layers': 2, 'lr': 0.005}
Best Validation Accuracy: 0.9545
